# LIBRERIAS

In [8]:
import pandas as pd
from google.cloud import bigquery
import pywhatkit
import time
import pyautogui
import time
from datetime import datetime, timedelta

# EXTRAYENDO DATA DE BIG QUERY Y CONVIRTIENDOLO EN DF

In [9]:
# Conexión Google Cloud SDK
# https://cloud.google.com/appengine/docs/standard/setting-up-environment?tab=python&hl=es-419

# Ejecutar lo siguiente en Powershell
## (New-Object Net.WebClient).DownloadFile("https://dl.google.com/dl/cloudsdk/channels/rapid/GoogleCloudSDKInstaller.exe", "$env:Temp\GoogleCloudSDKInstaller.exe") & $env:Temp\GoogleCloudSDKInstaller.exe
## gcloud auth application-default login

In [10]:
# Especifica el ID del proyecto de Google Cloud
project_id = 'bi-fcom-drmb-local-pe-sbx'

# Crea una instancia del cliente de BigQuery con el ID del proyecto especificado
client = bigquery.Client(project=project_id)

# ID del conjunto de datos que contiene la tabla
dataset_id = 'Snorlax_SellerJourney'

# Nombre de la tabla a la que deseas acceder
table_id = 'sellers_registered_ventas'

# Consulta SQL para obtener datos de la tabla
query = f"""
SELECT
  src_id,
  name,
  order_nr,
  shipping_provider_product,
  src_status,
  fecha_venta,
  target_to_ship,
  fecha_primera_venta,
  CONCAT('+51', Telefono) AS Telefono_Concat,
  origen,
  account_status
FROM (
  SELECT
    *
  FROM (
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY src_id ORDER BY fecha_venta ASC ) AS tt
    FROM (
      SELECT
        *
      FROM
        `bi-fcom-drmb-local-pe-sbx.Snorlax_SellerJourney.sellers_registered_ventas`
      WHERE
        src_status NOT IN( 'canceled',
          'return_shipped_by_customer',
          'failed_delivery',
          'return_waiting_for_approval',
          'return_rejected',
          'returned')))
  WHERE
    tt = 1)
WHERE
  fecha_venta BETWEEN '2024-03-05'AND '2024-03-31'
  AND origen ='nacional'
  AND account_status = 'Activo'
"""

# Ejecutar la consulta y obtener los resultados
query_job = client.query(query)
# Convertir los resultados a un DataFrame
df = query_job.to_dataframe()
print(df)

c:\Users\jalopezz\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


      src_id                        name    order_nr  \
0    SC71F50                DURANGO PERU  2177140044   
1    SC75600                 SANDDER TNT  2176225222   
2    SCC0AED            TUTTO COLLECTION  2174514253   
3    SC1E35A  ONLINE DISTRIBUTIONS PERU.  2177169166   
4    SC65C6E               JolkShop Peru  2175353098   
..       ...                         ...         ...   
112  SCD3FFE              MUEBLES WALTER  2178887479   
113  SC06F60   importaciones tukuy rikuy  2174828244   
114  SC634B5                   FullHouse  2180681239   
115  SC69546         ACUARIUM LOS OLIVOS  2179077128   
116  SC888DC                       DARNA  2175233824   

    shipping_provider_product             src_status         fecha_venta  \
0                        ibis              delivered 2024-03-11 03:56:54   
1                        ibis              delivered 2024-03-09 01:08:42   
2                        ibis              delivered 2024-03-05 12:03:37   
3                      

# DEFINIENDO EL DATA FRAME Y CAMPO FECHA_VENTA

In [11]:
# Crear DataFrame y especificar que columnas con tipo str
info_data = pd.DataFrame(df, columns=["src_id","name", "order_nr", "shipping_provider_product", "src_status", "fecha_venta","target_to_ship","fecha_primera_venta","Telefono_Concat"], dtype=str)
# Convertir fechas en formato de cadena a objetos datetime
info_data['fecha_venta'] = pd.to_datetime(info_data['fecha_venta'])

# DEFINIENDO PARAMETROS DE DÍAS Y HORAS

In [12]:
# Obtener la fecha de hoy y ayer
hoy = datetime.today().date()
ayer = hoy - timedelta(days=1)

# Establecer las horas límite para los envíos
if hoy.weekday() != 0:  # Si no es lunes
    hora_limite_am_inicio = ayer.strftime('%Y-%m-%d') + " 18:00:00"  # 6 PM de ayer
    hora_limite_am_fin = hoy.strftime('%Y-%m-%d') + " 08:00:00"       # 8 AM de hoy
else:  # Si es lunes
    viernes_pasado = hoy - timedelta(days=(hoy.weekday() + 3) % 7)  # Obtener el viernes pasado
    hora_limite_am_inicio = viernes_pasado.strftime('%Y-%m-%d') + " 18:00:00"  # 6 PM del viernes pasado
    hora_limite_am_fin = hoy.strftime('%Y-%m-%d') + " 08:00:00"                 # 8 AM de hoy

# Convertir fechas en formato de cadena a objetos datetime
info_data['fecha_venta'] = pd.to_datetime(info_data['fecha_venta'])

# CONDICIONALES 

In [13]:
# Filtrar las órdenes para el envío por la mañana
info_data_primera_venta_envioam = info_data[
    ((info_data['fecha_venta'] >= pd.to_datetime(hora_limite_am_inicio)) &
    (info_data['fecha_venta'] < pd.to_datetime(hora_limite_am_fin)))
]


print (info_data_primera_venta_envioam)

      src_id                           name    order_nr  \
19   SC5CFFD  INVERSIONES ALDOS PERU S.A.C.  2180727787   
25   SC1FCA5                       HAPPYPEE  2180645640   
36   SC77FAC                    Merary Perú  2180688365   
38   SC81A70                        MAHATMA  2180735506   
59   SC14EA3                         Bhamzz  2180639100   
67   SCA543E                INTERIORES PERU  2180826664   
85   SCCB615                   LuiNet Store  2180797994   
91   SCAF2F9                    JM PROJECTS  2180828819   
107  SC12FF3                      Xclusive.  2180780157   
114  SC634B5                      FullHouse  2180681239   

    shipping_provider_product             src_status         fecha_venta  \
19                       ibis  packed_by_marketplace 2024-03-18 23:30:34   
25                       ibis                shipped 2024-03-18 19:49:13   
36                       ibis          ready_to_ship 2024-03-18 21:41:12   
38                       ibis  packed_by_marke

# ENVIO WHATS APP

In [14]:
mensaje_whatsapp_am = """
¡Hola! *{Seller}* has recibido una venta 🎉 Queremos reforzar contigo el proceso para gestionarla:      
 
Tutorial despacho: https://youtu.be/lpk1_y9uYDg   
    
_*Instrucciones:*_  
 
Revisa tus órdenes aquí: https://sellercenter.falabella.com/order    
 
📦 Para gestionar tu pedido debes:      
 
1- Imprimir la *etiqueta* que pegarás al exterior de tu empaque. Se visualiza al lado izquierdo como *Label*. 
   
2- Hacer clic en *Listo para despachar* - máximo un día antes de la fecha límite (antes de las 8:00PM).   
   
3- El *{fecha_límite}* recolectaremos las órdenes en tu almacén 🤝  
 
🚨 Recuerda que:  
 
1- El horario de recolección es entre *8:00 a 17:00 hrs*. El transporte te llamará al llegar (espera: 10 minutos) 
 
2- Emitir la *boleta electrónica* en SUNAT por el "Gran total" y adjuntarla en Ordenes - Documentos Tributarios.
 
3- Instructivo de embalaje y etiquetado: https://www.falabellasellers.com/embalaje-y-etiquetado 
 
*4- Siempre enviamos las alertas de pedidos por correo, remitente: noreply@mail.sellercenter.falabella.com*
"""



for seller in info_data_primera_venta_envioam['src_id'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller = info_data_primera_venta_envioam[info_data_primera_venta_envioam['src_id'] == seller]
    
    
    Numero = df_seller['Telefono_Concat'].iloc[0]
    
    
    whatsapp_message = mensaje_whatsapp_am.format(
                    Seller=df_seller["name"].iloc[0],
                    fecha_límite=df_seller["target_to_ship"].iloc[0],                    
            )
    #Enviar el mensaje de WhatsApp
    pywhatkit.sendwhatmsg_instantly(Numero, whatsapp_message)
    time.sleep(10)    

KeyboardInterrupt: 